In [1]:
## python script to import/clean up second round (round 2 and round 3) MTurk data.
## generate input data for Stata analysis

## perform permutation test to check whether mean values from round 2 and 3 are different from mean values using round 1 data.


In [2]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np

In [7]:
# input round 2 and round 3 results

def data_input(filename):
    datain=pd.read_csv(r'C:\\Users\\jia\\Dropbox\\COVID-19\\working_home\\work\\raw_data\\'+filename+'_mturk_results.csv')
    datain['answer']=0*(datain['Answer.category.label']=='No')+0.5*(datain['Answer.category.label']=='Unknown')+1*(datain['Answer.category.label']=='Yes')
    datain['count']=1
    dataout=datain.groupby('Input.title').agg(
        sum_worktime=pd.NamedAgg(column='WorkTimeInSeconds', aggfunc=sum),
        sum_remote=pd.NamedAgg(column='answer', aggfunc=sum),
        count_answer=pd.NamedAgg(column='count', aggfunc=sum)  
    )
    data_temp=datain[['Input.title','WorkTimeInSeconds','Answer.category.label']].copy()
    data_temp['isco']=data_temp['Input.title'].str[0:4]
    data_temp['title']=data_temp['Input.title'].str[4:]
    data_temp['value']=(data_temp['Answer.category.label']=='Yes')*1+(data_temp['Answer.category.label']=='No')*0+(data_temp['Answer.category.label']=='Unknown')*0.5
    data_temp.to_csv(r'C:\\Users\\jia\\Dropbox\\COVID-19\\working_home\\work\\'+filename+'.csv')
    return [dataout, data_temp] 

[dataout_2, data_round2]=data_input('round_two')
[dataout_3, data_round3]=data_input('round_three')

# list of isco in round 2 and round 3

print("Number of occupations in round 2 and 3: ", data_round2.isco.unique().size)
list_isco=data_round2.isco.unique()

Number of occupations in round 2 and 3:  90


In [8]:
df = pd.read_feather('all_answers.feather')
df['value']=(df['answernum']==1)*1+(df['answernum']==2)*0+(df['answernum']==3)*0.5

list_isco_r1=df.ISCO.unique()
print("Number of occupations in round 1: ", list_isco_r1.size)

Number of occupations in round 1:  426


In [9]:
# permutation test for mean and median

def exact_mc_perm_test(xs, ys, nmc):
    n, k = len(xs), 0
    diff = np.abs(np.mean(xs) - np.mean(ys))
    zs = np.concatenate([xs, ys])
    for j in range(nmc):
        np.random.shuffle(zs)
        k += diff < np.abs(np.mean(zs[:n]) - np.mean(zs[n:]))
    return k / nmc

def exact_mc_perm_test_median(xs, ys, nmc):
    n, k = len(xs), 0
    diff = np.abs(np.median(xs) - np.median(ys))
    zs = np.concatenate([xs, ys])
    for j in range(nmc):
        np.random.shuffle(zs)
        k += diff < np.abs(np.median(zs[:n]) - np.median(zs[n:]))
    return k / nmc

In [10]:
# permutation test, round 1 vs round 2

pvalue=np.zeros([90,2])
i=0
for isco in list_isco:
    x=df[df['ISCO']==int(isco)]['value']
    y=data_round2[data_round2['isco']==isco]['value']
    pvalue[i,0]=int(isco)
    pvalue[i,1]=exact_mc_perm_test(x, y, 50000)
    if pvalue[i,1]<0.05:
        print(isco, pvalue[i,1])
    i += 1


1343 0.01932
2163 0.03928
2310 0.0
2423 0.02214
2634 0.0
3352 0.0211
3413 0.02254
4225 0.0
5312 0.0
6123 0.02146


In [11]:
# permutation test, round 1 vs (round 2 +round 3)   
pvalue=np.zeros([90,2])
i=0
for isco in list_isco:
    x=df[df['ISCO']==int(isco)]['value']
    x2=data_round2[data_round2['isco']==isco]['value']
    x3=data_round3[data_round3['isco']==isco]['value']
    y=x2.append(x3)
    pvalue[i,0]=int(isco)
    pvalue[i,1]=exact_mc_perm_test(x, y, 50000)
    if pvalue[i,1]<0.05:
        print(isco, pvalue[i,1])
    i += 1

1343 0.03968
2163 0.00512
2310 0.0
2355 0.01408
2423 0.03048
2434 0.03014
2522 0.03184
2634 0.0
2652 0.03176
2656 0.03216
3352 0.00796
3413 0.03194
4225 0.0
5223 0.03432
6122 0.0327


In [14]:
## all rounds in one dataframe

df['isco']=df['ISCO'].apply(str)
df_r1=df[['isco','value']].copy()
df_r2=data_round2[['isco','value']].copy()
df_r3=data_round3[['isco','value']].copy()
df_all=df_r1.append(df_r2).append(df_r3)
print(df_all)

     isco  value
0    7316    1.0
1    7316    1.0
2    7316    1.0
3    7316    1.0
4    7316    0.5
..    ...    ...
445  8156    1.0
446  8156    0.0
447  8156    0.0
448  8156    0.0
449  8156    0.0

[3975 rows x 2 columns]


In [15]:
def bootstrap_mean_sd(x, nmc):
    sample_mean=np.zeros(nmc)
    n = len(x)
    for j in range(nmc):
        sample_n=np.random.choice(x, size=n)
        sample_mean[j]=(sample_n.mean())
    return [x.mean(), sample_mean.std(), n]

In [16]:
## generate the standard errors for the labels for each occupation
## using the bootstrap method

list_isco_r1=df_all.isco.unique()
isco_n=list_isco_r1.size
pvalue=np.zeros([isco_n,4])
i=0
for isco in list_isco_r1:
    x=df_all[df_all['isco']==isco]['value']
    pvalue[i,0]=int(isco)
    [pvalue[i,1], pvalue[i,2], pvalue[i,3]]=bootstrap_mean_sd(x,10000)
    print("%4i, %4.3f,%4.3f, %2i" % (pvalue[i,0],pvalue[i,1],pvalue[i,2],pvalue[i,3])) 
    i +=1

7316, 0.900,0.089,  5
5322, 0.300,0.101, 20
3341, 1.000,0.000,  5
2113, 0.260,0.081, 25
8312, 0.000,0.000,  5
3512, 1.000,0.000,  5
9213, 0.000,0.000,  5
8154, 0.000,0.000,  5
8183, 0.200,0.180,  5
1221, 1.000,0.000, 10
7132, 0.000,0.000,  5
8343, 0.200,0.179,  5
3521, 0.475,0.108, 20
8331, 0.200,0.179,  5
2142, 0.000,0.000, 10
7318, 0.650,0.107, 20
3514, 0.800,0.178,  5
6130, 0.100,0.090,  5
2165, 0.500,0.111, 20
2434, 0.825,0.082, 20
7542, 0.225,0.089, 20
7521, 0.000,0.000,  5
8321, 0.000,0.000,  5
9321, 0.200,0.179,  5
1113, 0.200,0.179,  5
3433, 0.000,0.000,  5
7535, 0.100,0.090,  5
4413, 1.000,0.000,  5
3313, 0.800,0.178,  5
9214, 0.200,0.180,  5
5153, 0.000,0.000,  5
8143, 0.000,0.000,  5
2412, 1.000,0.000,  5
7513, 0.000,0.000,  5
2114, 0.200,0.127, 10
7534, 0.000,0.000,  5
1412, 0.300,0.102, 20
2264, 0.300,0.102, 20
3323, 0.800,0.178,  5
1322, 0.200,0.126, 10
9629, 0.000,0.000,  5
5329, 0.200,0.177,  5
5132, 0.200,0.180,  5
2621, 0.000,0.000,  5
2269, 0.000,0.000,  5
4311, 0.80

7211, 0.000,0.000,  5
8219, 0.200,0.177,  5
3142, 0.000,0.000,  5
5243, 0.000,0.000,  5
8151, 0.100,0.089,  5
2622, 0.350,0.107, 20
3117, 0.000,0.000,  5
2131, 0.200,0.126, 10
4227, 1.000,0.000,  5
4414, 1.000,0.000,  5
2654, 0.200,0.178,  5
5163, 0.000,0.000,  5
3339, 0.900,0.088,  5
7126, 0.000,0.000,  5
2413, 1.000,0.000,  5
7125, 0.200,0.178,  5
2265, 1.000,0.000,  5
3343, 0.800,0.178,  5
7323, 0.800,0.179,  5
3230, 0.200,0.179,  5
2356, 0.800,0.180,  5
7127, 0.000,0.000,  5
9212, 0.000,0.000,  5
3331, 0.400,0.111, 20
7115, 0.400,0.109, 20
2619, 0.300,0.180,  5
3334, 0.525,0.108, 20
3321, 1.000,0.000,  5
2522, 0.800,0.089, 20
4212, 0.800,0.178,  5
2162, 0.300,0.102, 20
9329, 0.200,0.180,  5
2355, 0.725,0.096, 20
8322, 0.000,0.000,  5
2511, 1.000,0.000,  5
3344, 0.575,0.107, 20
9215, 0.000,0.000,  5
7231, 0.000,0.000,  5
7541, 0.000,0.000,  5
9112, 0.000,0.000,  5
3253, 0.425,0.107, 20
9123, 0.200,0.177,  5
3214, 0.100,0.089,  5
3422, 0.000,0.000,  5
5411, 0.000,0.000,  5
3211, 0.20

In [17]:
## save the aggregated results into a csv file for stata use
## each line is one occupation

df_p=pd.DataFrame(pvalue)
df_p.to_csv(r'C:\\Users\\jia\\Dropbox\\COVID-19\\working_home\\work\\allthree.csv')